<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/20250112a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import math

# Load data
file_path = '/content/input/market_data.csv'
data = pd.read_csv(file_path)

# Initialize prior bar variables
data['PBH'] = data['high'].shift(1)
data['PBL'] = data['low'].shift(1)
data['PBC'] = data['close'].shift(1)

data['Range'] = data['high'] - data['low']
data['%R'] = (data['close'] - data['low']) / data['Range']

# CE is now based on the PRIOR BAR %R
data['Prior_%R'] = data['%R'].shift(1)
data = data.dropna(subset=['PBH', 'PBL', 'PBC', 'Prior_%R'])  # Remove rows without prior bar data

data['CE'] = np.where(data['Prior_%R'] >= 0.5, 1 - data['Prior_%R'], data['Prior_%R'])

def calculate_epc(ce):
    if ce == 0:
        return 1
    return min(max(math.ceil(ce / 0.1), 1), 5)

data['EPC'] = data['CE'].apply(calculate_epc)

data['Bias'] = np.where(data['CE'] == data['Prior_%R'], 'D', 'U')

data['REU'] = np.where(data['high'] > data['PBH'], data['PBH'] - data['high'], 0)
data['RED'] = np.where(data['low'] < data['PBL'], abs(data['PBL'] - data['low']), 0)

data['REU_Flag'] = np.where(data['REU'] == 0, 'NoRE', 'REU')
data['RED_Flag'] = np.where(data['RED'] == 0, 'NoRE', 'RED')

data['E1'] = np.where(data['Bias'] == 'U', data['REU'], data['RED'])
data['E2'] = np.where(data['Bias'] == 'U', data['RED'], data['REU'])

data['E1_Flag'] = np.where(data['E1'] == 0, 'NoE1', 'E1')
data['E2_Flag'] = np.where(data['E2'] == 0, 'NoE2', 'E2')

# Summary statistics
grouped = data.groupby(['Bias', 'EPC', 'E1_Flag']).size().unstack(fill_value=0)
grouped['%E1'] = grouped['E1'] / (grouped['E1'] + grouped['NoE1']) * 100
grouped['%NoE1'] = grouped['NoE1'] / (grouped['E1'] + grouped['NoE1']) * 100

print(grouped)

# Optional: Save summary to a CSV
grouped.to_csv('/content/output/summary_stats.csv')


E1_Flag    E1  NoE1        %E1      %NoE1
Bias EPC                                 
D    1    653   219  74.885321  25.114679
     2    481   212  69.408369  30.591631
     3    449   269  62.534819  37.465181
     4    410   292  58.404558  41.595442
     5    383   340  52.973721  47.026279
U    1    853   245  77.686703  22.313297
     2    575   230  71.428571  28.571429
     3    485   275  63.815789  36.184211
     4    410   302  57.584270  42.415730
     5    367   331  52.578797  47.421203
